In [1]:
!pip install pandas openpyxl xlrd

Defaulting to user installation because normal site-packages is not writeable


In [28]:

import pandas as pd

df = pd.read_excel(r"C:\Users\Admin\Downloads\PO4675.xlsx")

selected_columns = ['#', 'Item #', 'Vendor Item #', 'Description', 'Size', 'Quantity']
df_selected = df[selected_columns]
df_selected = df[selected_columns].copy() 
# Rename columns
df_selected.rename(columns={
    '#': 'SrNo',
    'Item #': 'SKU',
    'Vendor Item #': 'StyleCode',
    'Description': 'CustomerProductionInstruction',
    'PD#': 'ItemRefNo',
    'Description': 'CustomerProductionInstruction',
    'Size': 'ItemSize',
    'Quantity': 'OrderQty',
}, inplace=True)

# Move Stylecode column after SrNo
style_code = df_selected.pop('StyleCode')
df_selected.insert(df_selected.columns.get_loc('SrNo') + 1, 'StyleCode', style_code)

item_size = df_selected.pop('ItemSize')
df_selected.insert(df_selected.columns.get_loc('StyleCode') + 1, 'ItemSize', item_size)

order_qty = df_selected.pop('OrderQty')
df_selected.insert(df_selected.columns.get_loc('ItemSize') + 1, 'OrderQty', order_qty)

Cust_Prod_Inst = df_selected.pop('CustomerProductionInstruction')
df_selected.insert(df_selected.columns.get_loc('OrderQty') + 1, 'CustomerProductionInstruction', Cust_Prod_Inst)

# Add 'StampInstruction' and 'OrderGroup' columns after 'CustomerProductionInstruction'
stamp_instruction = input("Enter StampInstruction value: ")
order_group = input("Enter OrderGroup value: ")  #----helzberg 
df_selected.insert(df_selected.columns.get_loc('CustomerProductionInstruction') + 1, 'StampInstruction', stamp_instruction)
df_selected.insert(df_selected.columns.get_loc('CustomerProductionInstruction') + 2, 'OrderGroup', order_group)

# Add new column 'OrderItemPcs' after 'OrderQty'
df_selected.insert(df_selected.columns.get_loc('OrderQty') + 1, 'OrderItemPcs', value='')  # Initialize with 0 or empty values

# Function to map CustomerProductionInstruction to Metal
def map_metal(text):   #G14WZ IF RECYCLED(USER INPUT) PC95Z IF RECYCLED --------pending
    if pd.isna(text):
        return ''
    text = str(text).upper()
    if '14KY' in text:
        return 'G14Y'
    elif '14KW' in text:
        return 'G14W'
    elif '18KY' in text:
        return 'G18Y'
    elif '18KW' in text:
        return 'G18W'
    elif '10KY' in text:
        return 'G10Y'
    elif '10KW' in text:
        return 'G10W'
    elif 'PT' in text:
        return 'PC95'
    else:
        return ''

# Insert 'Metal' column after 'OrderItemPcs'
df_selected.insert(df_selected.columns.get_loc('OrderItemPcs') + 1, 'Metal', df_selected['CustomerProductionInstruction'].apply(map_metal))

# Function to create 'Tone' based on 'Metal'
def map_tone(metal):
    if metal == 'PC95':
        return 'PT'
    elif metal:
        return metal[-1]
    else:
        return ''

# Insert 'Tone' column after 'Metal'
df_selected.insert(df_selected.columns.get_loc('Metal') + 1, 'Tone', df_selected['Metal'].apply(map_tone))

# Ask user for ItemPoNo
#item_po_no = input("Enter ItemPoNo number: ")   ---------------temp disable

# Insert 'ItemPoNo' column after 'Tone'
df_selected.insert(df_selected.columns.get_loc('Tone') + 1, 'ItemPoNo', item_po_no)

# Add new columns 'ItemRefNo' and 'StockType' after 'ItemPoNo'
df_selected.insert(df_selected.columns.get_loc('ItemPoNo') + 1, 'ItemRefNo', '')
df_selected.insert(df_selected.columns.get_loc('ItemPoNo') + 2, 'StockType', '')

# Add 'Priority' column after 'StockType' with default '5 day' or user input
user_priority = input("Enter Priority (press Enter to use default '5 day'): ")
priority_value = user_priority if user_priority.strip() != "" else "5 day"
df_selected.insert(df_selected.columns.get_loc('StockType') + 1, 'Priority', priority_value)

#Add Make Type coln
df_selected.insert(df_selected.columns.get_loc('Priority') + 1, 'MakeType', '')

df_selected.head()

Enter StampInstruction value:  
Enter OrderGroup value:  
Enter Priority (press Enter to use default '5 day'):  


,SrNo,StyleCode,ItemSize,OrderQty,OrderItemPcs,Metal,Tone,ItemPoNo,ItemRefNo,StockType,Priority,MakeType,CustomerProductionInstruction,StampInstruction,OrderGroup,SKU
0,1,RG0001489H,6.5,9,,G14Y,Y,4656,,,5 day,,"WSM#41105493, 14KY SEMI 13D=1.066CTW 1RADINAT=...",,,RG1489H-LGD-14KY-SEMI-SH
1,2,RG0001489J,6.5,22,,G14Y,Y,4656,,,5 day,,"WSM#41105495, 14KY SEMI 13D=1.566CTW 1RADINAT=...",,,RG1489J-LGD-14KY-SEMI-SH
2,3,RG0001490H,6.5,15,,G14Y,Y,4656,,,5 day,,"WSM#41105497, 14KY SEMI 13D=1.066CTW 1OVAL=1.0...",,,RG1490H-LGD-14KY-SEMI-SH
3,4,RG0001490K,6.5,5,,G14Y,Y,4656,,,5 day,,"WSM#41105501, 14KY SEMI 13D=2.066CTW 1OVAL=2.0...",,,RG1490K-LGD-14KY-SEMI-SH
4,5,RG0001488H,6.5,4,,G14Y,Y,4656,,,5 day,,"WSM#41105508, 14KY SEMI 13D=1.066CTW 1HEXAGONE...",,,RG1488H-LGD-14KY-SH-SEMI


In [40]:
import pandas as pd

# --- Step 1: Read Excel ---
df = pd.read_excel(r"C:\Users\Admin\Downloads\PO4675.xlsx")

# --- Step 2: Select relevant columns ---
selected_columns = ['#', 'Item #', 'Vendor Item #', 'Description', 'Size', 'Quantity']
df = df[selected_columns].copy()

# --- Step 3: Rename columns ---
df.rename(columns={
    '#': 'SrNo',
    'Item #': 'SKU',
    'Vendor Item #': 'StyleCode',
    'Description': 'CustomerProductionInstruction',
    'Size': 'ItemSize',
    'Quantity': 'OrderQty',
}, inplace=True)

# --- Step 4: Ask user for input columns ---
item_po_no = input("Enter ItemPoNo number: ")
stamp_instruction = input("Enter StampInstruction value: ")
order_group = input("Enter OrderGroup value: ")
user_priority = input("Enter Priority (press Enter to use default '5 day'): ")
priority_value = user_priority if user_priority.strip() != "" else "5 day"

# --- Step 5: Functions for calculated columns ---
def map_metal(text):
    if pd.isna(text):
        return ''
    text = str(text).upper()
    if '14KY' in text:
        return 'G14Y'
    elif '14KW' in text:
        return 'G14W'
    elif '18KY' in text:
        return 'G18Y'
    elif '18KW' in text:
        return 'G18W'
    elif '10KY' in text:
        return 'G10Y'
    elif '10KW' in text:
        return 'G10W'
    elif 'PT' in text:
        return 'PC95'
    else:
        return ''

def map_tone(metal):
    if metal == 'PC95':
        return 'PT'
    elif metal:
        return metal[-1]
    else:
        return ''

def metal_text(metal):
    mapping = {
        "G14Y": "14K YELLOW GOLD",
        "G14W": "14K WHITE GOLD",
        "G18Y": "18K YELLOW GOLD",
        "G18W": "18K WHITE GOLD",
        "G10Y": "10K YELLOW GOLD",
        "G10W": "10K WHITE GOLD",
        "PC95": "PLATINUM"
    }
    return mapping.get(metal, "")

def extract_dia_quality(text):
    if pd.isna(text):
        return ""
    text = str(text).upper()
    if "LGD" in text:
        return text.split("LGD",1)[1].strip()
    return ""

def special_remarks(row):
    metal_descr = metal_text(row['Metal'])
    dia_quality = extract_dia_quality(row['CustomerProductionInstruction'])
    return f"{row['OrderGroup']},{row['SKU']},{metal_descr}, DIA QLTY - LGD {dia_quality}"

# --- Step 6: Create all new columns ---
df['OrderItemPcs'] = ''
df['Metal'] = df['CustomerProductionInstruction'].apply(map_metal)
df['Tone'] = df['Metal'].apply(map_tone)
df['ItemPoNo'] = item_po_no
df['ItemRefNo'] = ''
df['StockType'] = ''
df['Priority'] = priority_value
df['StampInstruction'] = stamp_instruction
df['OrderGroup'] = order_group
df['SpecialRemarks'] = df.apply(special_remarks, axis=1)

# --- Step 7: Define final column order ---
final_columns = [
    'SrNo', 'StyleCode', 'ItemSize', 'OrderQty', 'OrderItemPcs', 'Metal', 'Tone',
    'ItemPoNo', 'ItemRefNo', 'StockType', 'Priority',
    'CustomerProductionInstruction', 'StampInstruction', 'OrderGroup', 'SpecialRemarks', 'SKU'
]

df = df[final_columns]

# --- Step 8: Preview final DataFrame ---
df.to_csv(r'C:\Users\Admin\Desktop\Bhakti_PO_Cleaned.csv', index=False)
df.head()


Enter ItemPoNo number:  4675
Enter StampInstruction value:  
Enter OrderGroup value:  Helzberg
Enter Priority (press Enter to use default '5 day'):  


,SrNo,StyleCode,ItemSize,OrderQty,OrderItemPcs,Metal,Tone,ItemPoNo,ItemRefNo,StockType,Priority,CustomerProductionInstruction,StampInstruction,OrderGroup,SpecialRemarks,SKU
0,1,RG0001489H,6.5,9,,G14Y,Y,4675,,,5 day,"WSM#41105493, 14KY SEMI 13D=1.066CTW 1RADINAT=...",,Helzberg,Helzberg | RG1489H-LGD-14KY-SEMI-SH | 14K YELL...,RG1489H-LGD-14KY-SEMI-SH
1,2,RG0001489J,6.5,22,,G14Y,Y,4675,,,5 day,"WSM#41105495, 14KY SEMI 13D=1.566CTW 1RADINAT=...",,Helzberg,Helzberg | RG1489J-LGD-14KY-SEMI-SH | 14K YELL...,RG1489J-LGD-14KY-SEMI-SH
2,3,RG0001490H,6.5,15,,G14Y,Y,4675,,,5 day,"WSM#41105497, 14KY SEMI 13D=1.066CTW 1OVAL=1.0...",,Helzberg,Helzberg | RG1490H-LGD-14KY-SEMI-SH | 14K YELL...,RG1490H-LGD-14KY-SEMI-SH
3,4,RG0001490K,6.5,5,,G14Y,Y,4675,,,5 day,"WSM#41105501, 14KY SEMI 13D=2.066CTW 1OVAL=2.0...",,Helzberg,Helzberg | RG1490K-LGD-14KY-SEMI-SH | 14K YELL...,RG1490K-LGD-14KY-SEMI-SH
4,5,RG0001488H,6.5,4,,G14Y,Y,4675,,,5 day,"WSM#41105508, 14KY SEMI 13D=1.066CTW 1HEXAGONE...",,Helzberg,Helzberg | RG1488H-LGD-14KY-SH-SEMI | 14K YELL...,RG1488H-LGD-14KY-SH-SEMI


In [43]:
import pandas as pd

# --- Step 1: Read Excel ---
df = pd.read_excel(r"C:\Users\Admin\Downloads\PO4675.xlsx")

# --- Step 2: Select relevant columns ---
selected_columns = ['#', 'Item #', 'Vendor Item #', 'Description', 'Size', 'Quantity']
df = df[selected_columns].copy()

# --- Step 3: Rename columns ---
df.rename(columns={
    '#': 'SrNo',
    'Item #': 'SKU',
    'Vendor Item #': 'StyleCode',
    'Description': 'CustomerProductionInstruction',
    'Size': 'ItemSize',
    'Quantity': 'OrderQty',
}, inplace=True)

# --- Step 4: Ask user for input columns ---
item_po_no = input("Enter ItemPoNo number: ")
stamp_instruction = input("Enter StampInstruction value: ")
order_group = input("Enter OrderGroup value: ")
user_priority = input("Enter Priority (press Enter to use default '5 day'): ")
priority_value = user_priority if user_priority.strip() != "" else "5 day"

# --- Step 5: Functions for calculated columns ---
def map_metal(text):
    if pd.isna(text):
        return ''
    text = str(text).upper()
    if '14KY' in text:
        return 'G14Y'
    elif '14KW' in text:
        return 'G14W'
    elif '18KY' in text:
        return 'G18Y'
    elif '18KW' in text:
        return 'G18W'
    elif '10KY' in text:
        return 'G10Y'
    elif '10KW' in text:
        return 'G10W'
    elif 'PT' in text:
        return 'PC95'
    else:
        return ''

def map_tone(metal):
    if metal == 'PC95':
        return 'PT'
    elif metal:
        return metal[-1]
    else:
        return ''

def metal_text(metal):
    mapping = {
        "G14Y": "14K YELLOW GOLD",
        "G14W": "14K WHITE GOLD",
        "G18Y": "18K YELLOW GOLD",
        "G18W": "18K WHITE GOLD",
        "G10Y": "10K YELLOW GOLD",
        "G10W": "10K WHITE GOLD",
        "PC95": "PLATINUM"
    }
    return mapping.get(metal, "")

def extract_dia_quality(text):
    if pd.isna(text):
        return ""
    text = str(text).upper()
    if "LGD" in text:
        after_lgd = text.split("LGD", 1)[1].strip()
        # Take only first two words
        words = after_lgd.split()
        return ' '.join(words[:2])
    return ""

def special_remarks(row):
    metal_descr = metal_text(row['Metal'])
    dia_quality = extract_dia_quality(row['CustomerProductionInstruction'])
    return f"{row['OrderGroup']},{row['SKU']},{metal_descr}, DIA QLTY - {dia_quality}"

def design_production_instruction(row):
    tone = str(row['Tone']).upper()
    cpi = str(row['CustomerProductionInstruction']).upper() if pd.notna(row['CustomerProductionInstruction']) else ''
    semi_present = "SEMI" in cpi

    if tone == "W" and semi_present:
        return "SEMI MOUNT, WHITE RODIUM"
    elif tone == "Y" and semi_present:
        return "SEMI MOUNT, NO RODIUM"
    elif tone == "W" and not semi_present:
        return "WHITE RODIUM"
    elif tone == "Y" and not semi_present:
        return "NO RODIUM"
    elif tone == "PT" and semi_present:
        return "SEMI MOUNT, NO RODIUM"
    elif tone == "PT" and not semi_present:
        return "NO RODIUM"
    else:
        return ""

# --- Step 6: Create all new columns ---
df['OrderItemPcs'] = ''
df['Metal'] = df['CustomerProductionInstruction'].apply(map_metal)
df['Tone'] = df['Metal'].apply(map_tone)
df['ItemPoNo'] = item_po_no
df['ItemRefNo'] = ''
df['StockType'] = ''
df['Priority'] = priority_value
df['StampInstruction'] = stamp_instruction
df['OrderGroup'] = order_group
df['SpecialRemarks'] = df.apply(special_remarks, axis=1)
df['DesignProductionInstruction'] = df.apply(design_production_instruction, axis=1)

# --- Step 7: Reorder columns ---
cols = df.columns.tolist()
# Place SpecialRemarks after CustomerProductionInstruction
cust_idx = cols.index('CustomerProductionInstruction')
cols.remove('SpecialRemarks')
cols.insert(cust_idx + 1, 'SpecialRemarks')
# Place DesignProductionInstruction after SpecialRemarks
special_idx = cols.index('SpecialRemarks')
cols.remove('DesignProductionInstruction')
cols.insert(special_idx + 1, 'DesignProductionInstruction')
df = df[cols]

# --- Step 8: Export to CSV ---
df.to_csv(r'C:\Users\Admin\Desktop\Bhakti_PO_Cleaned.csv', index=False)

# --- Step 9: Preview final DataFrame ---
df.head()


Enter ItemPoNo number:  4675
Enter StampInstruction value:  
Enter OrderGroup value:  Helzberg
Enter Priority (press Enter to use default '5 day'):  


,SrNo,SKU,StyleCode,CustomerProductionInstruction,SpecialRemarks,DesignProductionInstruction,ItemSize,OrderQty,OrderItemPcs,Metal,Tone,ItemPoNo,ItemRefNo,StockType,Priority,StampInstruction,OrderGroup
0,1,RG1489H-LGD-14KY-SEMI-SH,RG0001489H,"WSM#41105493, 14KY SEMI 13D=1.066CTW 1RADINAT=...","Helzberg,RG1489H-LGD-14KY-SEMI-SH,14K YELLOW G...","SEMI MOUNT, NO RODIUM",6.5,9,,G14Y,Y,4675,,,5 day,,Helzberg
1,2,RG1489J-LGD-14KY-SEMI-SH,RG0001489J,"WSM#41105495, 14KY SEMI 13D=1.566CTW 1RADINAT=...","Helzberg,RG1489J-LGD-14KY-SEMI-SH,14K YELLOW G...","SEMI MOUNT, NO RODIUM",6.5,22,,G14Y,Y,4675,,,5 day,,Helzberg
2,3,RG1490H-LGD-14KY-SEMI-SH,RG0001490H,"WSM#41105497, 14KY SEMI 13D=1.066CTW 1OVAL=1.0...","Helzberg,RG1490H-LGD-14KY-SEMI-SH,14K YELLOW G...","SEMI MOUNT, NO RODIUM",6.5,15,,G14Y,Y,4675,,,5 day,,Helzberg
3,4,RG1490K-LGD-14KY-SEMI-SH,RG0001490K,"WSM#41105501, 14KY SEMI 13D=2.066CTW 1OVAL=2.0...","Helzberg,RG1490K-LGD-14KY-SEMI-SH,14K YELLOW G...","SEMI MOUNT, NO RODIUM",6.5,5,,G14Y,Y,4675,,,5 day,,Helzberg
4,5,RG1488H-LGD-14KY-SH-SEMI,RG0001488H,"WSM#41105508, 14KY SEMI 13D=1.066CTW 1HEXAGONE...","Helzberg,RG1488H-LGD-14KY-SH-SEMI,14K YELLOW G...","SEMI MOUNT, NO RODIUM",6.5,4,,G14Y,Y,4675,,,5 day,,Helzberg


In [55]:
#FINAL CODE
import pandas as pd

# --- Step 1: Read Excel ---
#df = pd.read_excel(r"C:\Users\Admin\Downloads\PO4675.xlsx")
df = pd.read_excel(r"C:\Users\Admin\Downloads\PO_BHAKTI_4656.xlsx")   # BHAKTI
#df = pd.read_excel(r"C:\Users\Admin\Downloads\VIMCO_PO.xlsx")  # VIMCO

# --- Step 2: Select relevant columns ---
selected_columns = ['#', 'Item #', 'Vendor Item #', 'Description', 'Size', 'Quantity']
df = df[selected_columns].copy()

# --- Step 3: Rename columns ---
df.rename(columns={
    '#': 'SrNo',
    'Item #': 'SKU',
    'Vendor Item #': 'StyleCode',
    'Description': 'CustomerProductionInstruction',
    'Size': 'ItemSize',
    'Quantity': 'OrderQty',
}, inplace=True)

# --- Step 4: Ask user for input columns ---
item_po_no = input("Enter ItemPoNo number: ")
stamp_instruction = input("Enter StampInstruction value: ")
order_group = input("Enter OrderGroup value: ")
user_priority = input("Enter Priority (press Enter to use default '5 day'): ")
po_no_value = input("Enter PO NO. value: ")
priority_value = user_priority if user_priority.strip() != "" else "5 day"

# --- Step 5: Functions for calculated columns ---
def map_metal(text):
    if pd.isna(text):
        return ''
    text = str(text).upper()
    if '14KY' in text:
        return 'G14Y'
    elif '14KW' in text:
        return 'G14W'
    elif '18KY' in text:
        return 'G18Y'
    elif '18KW' in text:
        return 'G18W'
    elif '10KY' in text:
        return 'G10Y'
    elif '10KW' in text:
        return 'G10W'
    elif 'PT' in text:
        return 'PC95'
    else:
        return ''

def map_tone(metal):
    if metal == 'PC95':
        return 'PT'
    elif metal:
        return metal[-1]
    else:
        return ''

def metal_text(metal):
    mapping = {
        "G14Y": "14K YELLOW GOLD",
        "G14W": "14K WHITE GOLD",
        "G18Y": "18K YELLOW GOLD",
        "G18W": "18K WHITE GOLD",
        "G10Y": "10K YELLOW GOLD",
        "G10W": "10K WHITE GOLD",
        "PC95": "PLATINUM"
    }
    return mapping.get(metal, "")

def extract_dia_quality(text):
    if pd.isna(text):
        return ""
    text = str(text).upper()
    if "LGD" in text:
        after_lgd = text.split("LGD", 1)[1].strip()
        words = after_lgd.split()
        return ' '.join(words[:2])
    return ""

def special_remarks(row):
    metal_descr = metal_text(row['Metal'])
    dia_quality = extract_dia_quality(row['CustomerProductionInstruction'])
    return f"{row['OrderGroup']},{row['SKU']},{metal_descr}, DIA QLTY - {dia_quality}"

def design_production_instruction(row):
    tone = str(row['Tone']).upper()
    cpi = str(row['CustomerProductionInstruction']).upper() if pd.notna(row['CustomerProductionInstruction']) else ''
    semi_present = "SEMI" in cpi

    if tone == "W" and semi_present:
        return "SEMI MOUNT, WHITE RODIUM"
    elif tone == "Y" and semi_present:
        return "SEMI MOUNT, NO RODIUM"
    elif tone == "W" and not semi_present:
        return "WHITE RODIUM"
    elif tone == "Y" and not semi_present:
        return "NO RODIUM"
    elif tone == "PT" and semi_present:
        return "SEMI MOUNT, NO RODIUM"
    elif tone == "PT" and not semi_present:
        return "NO RODIUM"
    else:
        return ""

# --- Step 6: Create all new columns ---
df['OrderItemPcs'] = ''
df['Metal'] = df['CustomerProductionInstruction'].apply(map_metal)
df['Tone'] = df['Metal'].apply(map_tone)
df['ItemPoNo'] = item_po_no
df['ItemRefNo'] = ''
df['StockType'] = ''
df['Priority'] = priority_value
df['MakeType'] = ''
df['StampInstruction'] = stamp_instruction
df['OrderGroup'] = order_group
df['Certificate'] = '' 
df['SpecialRemarks'] = df.apply(special_remarks, axis=1)
df['DesignProductionInstruction'] = df.apply(design_production_instruction, axis=1)
# --- Step 6c: Add new columns after SKU ---
new_cols_after_sku = [
    'Basestoneminwt', 'Basestonemaxwt', 'Basemetalminwt', 'Basemetalmaxwt',
    'Productiondeliverydate', 'Expecteddeliverydate', 'SetPrice', 'StoneQuality','PO NO.'
]

for col in new_cols_after_sku:
    if col == 'PO NO.':
        df[col] = po_no_value  # use user input for PO NO.
    else:
        df[col] = ''  # Initialize others empty

for col in new_cols_after_sku:
    df[col] = ''  # Initialize empty or default value
    
df.loc[df['StyleCode'].astype(str).str.upper().str.startswith('ZR'), 'ItemSize'] = 7 #LOGIC FOR ZR STYLEBY-DEFUALT BE 7 AS ITEMSIZE

def extract_size_from_sku(sku):  # logic for SZ9 FOR ITEMSIZE
    if pd.isna(sku):
        return None
    sku = str(sku).upper()
    match = re.search(r'SZ(\d+(\.\d+)?)$', sku)  # matches SZ9, SZ6.5, etc.
    if match:
        return match.group(1)
    return None

df['ExtractedSize'] = df['SKU'].apply(extract_size_from_sku)

# Update ItemSize with extracted value (only if not already set)
df['ItemSize'] = df['ItemSize'].fillna(df['ExtractedSize'])

# Drop helper column
df.drop(columns=['ExtractedSize'], inplace=True)

# --- Step 7: Reorder columns as requested ---
final_columns = [
    'SrNo', 'StyleCode', 'ItemSize', 'OrderQty', 'OrderItemPcs', 'Metal', 'Tone',
    'ItemPoNo', 'ItemRefNo', 'StockType', 'Priority', 'MakeType',
    'CustomerProductionInstruction', 'SpecialRemarks', 'DesignProductionInstruction',
    'StampInstruction', 'OrderGroup', 'Certificate', 'SKU'
] + new_cols_after_sku
df = df[final_columns]

# --- Step 8: Export to CSV ---
df.to_csv(r'C:\Users\Admin\Desktop\Bhakti_PO4656_Cleaned.csv', index=False)

# --- Step 9: Preview final DataFrame ---
df.head()


Enter ItemPoNo number:  4656
Enter StampInstruction value:  BHJ HDS LGD 14K
Enter OrderGroup value:  Helzberg
Enter Priority (press Enter to use default '5 day'):  


,SrNo,StyleCode,ItemSize,OrderQty,OrderItemPcs,Metal,Tone,ItemPoNo,ItemRefNo,StockType,...,Certificate,SKU,Basestoneminwt,Basestonemaxwt,Basemetalminwt,Basemetalmaxwt,Productiondeliverydate,Expecteddeliverydate,SetPrice,StoneQuality
0,1,RG0001964QA,9,1,,G14W,W,4656,,,...,,RG1964Q-LGD-14KW-SZ9,,,,,,,,
1,2,ZR0001964QA,7.0,1,,G14W,W,4656,,,...,,ZR1964Q-LGD-14KW-SZ9,,,,,,,,
